<a href="https://colab.research.google.com/github/SiwawawaXD/LLMclassification/blob/main/pythonCallChatgpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
# !pip install -q openai

In [18]:
!openai migrate




Analyzing Finding files                                                         
Processed 0 files and found 0 matches


In [24]:
import os
from openai import OpenAI
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

# Initialize client with your API key from Colab userdata
client = OpenAI(
   api_key=os.environ.get("OPENAI_API_KEY")
)

# Send request using chat.completions
response = client.chat.completions.create(
    model="gpt-4o-mini",  # you can also try "gpt-4o-mini" if gpt-5 is overloaded
    messages=[
        {"role": "system", "content": "You are a coding assistant that talks like a pirate."},
        {"role": "user", "content": "How do I check if a Python object is an instance of a class?"},
    ],temperature=0.5,
    max_tokens=1024
)

# Print the assistant's reply
print(response.choices[0].message.content)


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}